In [1]:
# !pip install tqdm

In [2]:
import pandas as pd
import tarfile
from tqdm import tqdm, tqdm_notebook, tnrange
import json
import numpy as np
import time
from copy import deepcopy

In [3]:
## The restaurants in the five candidate cities
df_res_bus = pd.read_csv('../datasets/comunidades/business_community_1.csv')
reviews = pd.read_csv('../datasets/comunidades/review_community_1.csv')
users = pd.read_csv('../datasets/comunidades/user_community_1.csv')


In [4]:
df_res_bus = df_res_bus.rename(columns={"name":"restaurant"})

In [5]:
df_res_bus.head()

,business_id,restaurant,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,XyGRDrgCK0z4CiA6nhwEaw,Persian Room Fine Dining,"9290 N Thornydale Rd, Ste 100",Tucson,AZ,85742,32.375016,-111.046320,4.5,258,1,"{'Alcohol': ""'full_bar'"", 'GoodForMeal': ""{'de...","Restaurants, Persian/Iranian, Mediterranean","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3..."
1,5Ce3lZksYVkCbrihqylVHQ,Sushi Nara,64 N Harrison Rd,Tucson,AZ,85748,32.222552,-110.788132,4.5,159,1,"{'RestaurantsPriceRange2': '2', 'HasTV': 'True...","Sushi Bars, Japanese, Restaurants","{'Monday': '16:30-20:0', 'Tuesday': '16:30-20:..."
2,xwKYBPO0ByGlkvNcr8FdqQ,Le Rendez-vous,3844 E Fort Lowell Rd,Tucson,AZ,85716,32.264930,-110.910544,4.0,223,1,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, French, Tapas/Small Plates","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
3,SVJJIoAiW3vHwoSpVnUcLQ,Pastiche Modern Eatery,3025 N Campbell Ave,Tucson,AZ,85719,32.261920,-110.944474,3.5,313,1,"{'WiFi': ""'free'"", 'HasTV': 'True', 'Restauran...","American (New), American (Traditional), Restau...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ..."
4,p7tSGmWpFV246aYhwODgBg,O Sushi Restaurant,4689 E Speedway Blvd,Tucson,AZ,85712,32.236944,-110.892965,4.5,199,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Sushi Bars, Restaurants, Japanese","{'Monday': '16:30-20:30', 'Tuesday': '16:30-20..."


In [6]:
reviews = reviews.rename(columns={"stars":"rating"})


In [7]:
reviews.head()

,review_id,user_id,business_id,rating,useful,funny,cool,text,date
0,ymhbOMW63B_vGaRFR3XT0A,yZdAhQ_KmKuCZmbBy9YDQg,5Ce3lZksYVkCbrihqylVHQ,5,0,0,0,I just started going to Sushi Nara this month ...,2014-07-25 17:56:26
1,9ntCEU8M_YBa0TIxxM1roA,BMXA8HiAuOQboRikPt6Xgg,XyGRDrgCK0z4CiA6nhwEaw,3,3,2,0,Ok. I have had Persian food before as it is on...,2018-07-17 02:44:51
2,KubhmZ6MADKfAh0TucXZeQ,iW4scl2ezCV6IGfOk6OtyQ,xwKYBPO0ByGlkvNcr8FdqQ,5,0,0,0,"Always outstanding. This evening, they were a ...",2017-04-30 03:57:53
3,jT0mpPypQH_eFoyIHhuM6A,To1Pq4io8oCT8f6ryRdPeQ,5Ce3lZksYVkCbrihqylVHQ,5,2,0,3,My kitty and I love this sushi place!! We love...,2015-10-07 23:33:00
4,mr8-n6rmPXqgAe0O5YG0jQ,4RZJaKdymRQ5z59ZMFznTQ,xwKYBPO0ByGlkvNcr8FdqQ,5,2,0,1,I took my gf for our one year anniversary here...,2014-09-10 04:16:53


In [8]:
#number of unique users and restaurants
n_users = len(reviews.user_id.unique())
n_restaurants = len(reviews.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 5598, unique restaurants: 34


In [9]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,1sGYXSkJHPhJ6wQtc-RbZw,Kimberly,820,2009-07-12 01:03:14,5441,2976,4264,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","9w4vI3LRTA4JdJyzOxeEMw, fNSGyM1NJ91CTO_VfJ5asQ...",149,...,31,43,12,8,154,403,541,541,265,148
1,2YKkLFeOx-0zRcWp0KUv_Q,Mark,564,2010-06-07 21:08:02,2158,1026,1068,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2...","qumjKpb34tsSJ26Y2aaTPg, fHxHETYCW-hJhPrEVNN3hA...",63,...,11,3,2,17,102,288,260,260,126,6
2,5BpJEC8YVi80ecWT5CtlfQ,Jenny,2739,2008-09-11 01:51:00,17333,11441,14336,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","nqyyLmjNdS22H0yKSu6-YQ, eVUmxx1kN-Acni0t7gcg3g...",427,...,190,168,52,75,508,1564,2415,2415,1035,1139
3,kF2sEC2B1nJlxVFr_jaEQA,Lara,198,2009-02-06 16:15:32,305,56,121,"2010,2011","-MRW5b_OiA4833cL4v53jQ, eW-IQGFyOcqg-aKuhxKf3g...",10,...,3,2,0,0,14,17,14,14,8,1
4,zdw1jwmw1TNLZgu_is_Seg,Pam,108,2011-03-21 03:27:57,214,52,85,"2011,2012,2013","fwOSFdqvyvG4B-jNaseceQ, 0ED7A-Tae6OKYL1hkD4tSA...",8,...,0,0,1,0,15,2,18,18,5,2


Train Test Split

In [10]:
SAMPLING_RATE = 1/5

In [11]:
# Downsample by users
user_id_unique = reviews.user_id.unique()
user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']) \
    .sample(frac= SAMPLING_RATE, replace=False, random_state=1)

reviews_sample = reviews.merge(user_id_sample, left_on='user_id', right_on='unique_user_id') \
   .drop(['unique_user_id'], axis=1)
reviews_sample.head()
#print(reviews_sample.shape)

,review_id,user_id,business_id,rating,useful,funny,cool,text,date
0,mr8-n6rmPXqgAe0O5YG0jQ,4RZJaKdymRQ5z59ZMFznTQ,xwKYBPO0ByGlkvNcr8FdqQ,5,2,0,1,I took my gf for our one year anniversary here...,2014-09-10 04:16:53
1,36eovmCnRloGn8oEOV5KrA,7Yy-wEDqUWYUGNQHkJ_Jcg,xwKYBPO0ByGlkvNcr8FdqQ,4,0,0,0,Their food was excellent! I was surprised it w...,2014-04-19 16:17:23
2,1a4BtVOSqEKIr3zZfZZFBg,zAFgnKwrygxSp1kXzdvMMA,XyGRDrgCK0z4CiA6nhwEaw,3,0,0,0,My family and I were super excited that Persia...,2018-07-17 15:26:09
3,RkpAuBy0Oi-zqGvfh76Tuw,fGYAraOy6CFZeg8-YRyMxw,5Ce3lZksYVkCbrihqylVHQ,5,3,0,0,"Great bang for your buck, sushi was fresh and ...",2018-06-09 03:48:32
4,s7gRAfoy33vvsKHzqIf-bQ,dXmmyBCImZmzBwqZ6nh4hg,xwKYBPO0ByGlkvNcr8FdqQ,5,3,0,1,We came with the meetup.com group and the serv...,2015-09-12 02:53:48


In [12]:
#number of unique users and restaurants
n_users = len(reviews_sample.user_id.unique())
n_restaurants = len(reviews_sample.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 1120, unique restaurants: 34


In [13]:
# hold out last review
reviews_user_date = reviews_sample.loc[:, ['user_id', 'date']]
reviews_user_date.date = pd.to_datetime(reviews_user_date.date)
index_holdout = reviews_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == reviews_user_date['date']
reviews_holdout_ = reviews_sample[index_holdout]
reviews_traincv_ = reviews_sample[~index_holdout]

reviews_user_date = reviews_traincv_.loc[:, ['user_id', 'date']]
index_holdout = reviews_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == reviews_user_date['date']
reviews_cv_ = reviews_traincv_[index_holdout]
reviews_train_ = reviews_traincv_[~index_holdout]

# remove the user that has fake reviews 

cv_users_del = set(reviews_cv_.user_id) - set(reviews_train_.user_id)
holdout_users_del = set(reviews_holdout_.user_id) - set(reviews_train_.user_id)
reviews_cv_ = reviews_cv_[~reviews_cv_.user_id.isin(cv_users_del)]
reviews_holdout_ = reviews_holdout_[~reviews_holdout_.user_id.isin(holdout_users_del)]

# ratings_cv_ = ratings_cv_[~ratings_cv_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]
# ratings_holdout_ = ratings_holdout_[~ratings_holdout_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]

print('There are {0} rows, {1} columns in training set.'.format(reviews_train_.shape[0], reviews_train_.shape[1]))
print('There are {0} rows, {1} columns in training set.'.format(reviews_cv_.shape[0], reviews_cv_.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(reviews_holdout_.shape[0], reviews_holdout_.shape[1]))


There are 126 rows, 9 columns in training set.
There are 58 rows, 9 columns in training set.
There are 58 rows, 9 columns in holdout set.


In [14]:
reviews_cv_2=reviews_cv_
reviews_cv_2.head()

,review_id,user_id,business_id,rating,useful,funny,cool,text,date
18,4bkjMeCA_kXEbLx9Qk6f_Q,s3qxdQoNyVd2Q_Ub9HstXg,xwKYBPO0ByGlkvNcr8FdqQ,5,5,0,1,The four most recent reviews have been far fro...,2010-01-07 04:07:01
41,FJHHC0uSPzsNT1RlFMAdKw,P2FNLDMpSpdo92ydQlw7-g,wzPc5d1l0PpxQLQDhh8T2w,1,9,7,2,I spent a long time looking forward to Toro Lo...,2019-08-08 20:15:42
50,nFyTc29sBNtyG0wm_RwijA,_Y28vILK64Nh5FpKom-GHA,IJ5N0TNgDDFP0UauRmepDg,4,1,0,0,This is a great little place with really delic...,2015-01-25 02:55:06
54,Z9F2Krv-WOAgGu46ooCGGg,srhTOSgTBtvHeISgwd_mgw,l8yqThofpWGUs6lxOeIdOA,2,1,0,1,I see this place still hasn't changed their wa...,2019-10-22 13:43:27
66,ODBg2SqECdigos5RuP40Mw,ZZPaQmmnEiL1fyN-4qtvJg,CJoO4HYD0tZRXlZqA04wmw,4,4,5,3,They know what I want when I walked in the doo...,2013-06-24 05:01:09


In [15]:
# check if we have a enough user sample size (> 3000)
number_of_unique_users = len(reviews_train_.user_id.unique())
print(number_of_unique_users)

58


Evaluation Metrics

In [16]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(df_res_bus, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                   'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

reviews_train = process(reviews_train_.copy())
reviews_holdout = process(reviews_holdout_.copy())
reviews_val = process(reviews_cv_.copy())

In [17]:
reviews_train_final = reviews_train.append(reviews_val)
reviews_entire_df = reviews_train.append(reviews_val).append(reviews_holdout)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_16920\2588244626.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviews_train_final = reviews_train.append(reviews_val)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_16920\2588244626.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviews_entire_df = reviews_train.append(reviews_val).append(reviews_holdout)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_16920\2588244626.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviews_entire_df = reviews_train.append(reviews_val).append(reviews_holdout)


In [18]:
reviews_entire_df.head()


,review_id,user_id,business_id,rating,useful_x,funny_x,cool_x,text,date,week_day,...,state,postal_code,latitude,longitude,stars,review_count_y,is_open,attributes,categories,hours
0,qx8317ZjR3A6VkEwmNTBQw,s3qxdQoNyVd2Q_Ub9HstXg,xwKYBPO0ByGlkvNcr8FdqQ,5,2,0,2,"Tucson's best restaurant, and has been for at ...",2009-08-13 22:35:41,3,...,AZ,85716,32.264930,-110.910544,4.0,223,1,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, French, Tapas/Small Plates","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
1,ItUWjCPGZMzPwnBYJWvIMA,P2FNLDMpSpdo92ydQlw7-g,xwKYBPO0ByGlkvNcr8FdqQ,5,6,0,0,"I came to Le Rendez-vous for my birthday, and ...",2015-03-31 16:48:35,1,...,AZ,85716,32.264930,-110.910544,4.0,223,1,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, French, Tapas/Small Plates","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
2,yUG9pDv7RUfmfwTPpoU0rg,Qaa2sxNfZyTUcmu2hpGTgw,xwKYBPO0ByGlkvNcr8FdqQ,5,3,0,0,We just went here for my boyfriends birthday. ...,2015-08-26 05:24:49,2,...,AZ,85716,32.264930,-110.910544,4.0,223,1,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, French, Tapas/Small Plates","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
3,PMrXIWpT3r4fH4HKUlMMYA,jL41h4Ez98PZfqiKzbbY1w,xwKYBPO0ByGlkvNcr8FdqQ,4,4,1,1,"Let's start with dessert, the best place for a...",2019-07-23 23:09:26,1,...,AZ,85716,32.264930,-110.910544,4.0,223,1,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, French, Tapas/Small Plates","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
4,jgNeBmzXFT0TTw0J4AJ3CA,P2FNLDMpSpdo92ydQlw7-g,LXR1f2ayOSq7w-Ux7UPUtg,5,2,0,1,Delicious sushi on the east side. The price se...,2014-02-08 22:23:24,5,...,AZ,85748,32.222546,-110.788133,3.5,103,0,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Japanese, Sushi Bars, Restaurants","{'Monday': '11:30-21:30', 'Tuesday': '11:30-21..."


In [19]:
unique_city_businesses = reviews_entire_df[['city','business_id']].drop_duplicates()
unique_cities = unique_city_businesses.groupby('city').count()['business_id']
#unique_cities = unique_cities[unique_cities > 100]
out = pd.DataFrame()
for city in unique_cities.index:
    tmp = reviews_holdout[(reviews_holdout['city'] ==city) &
                          (reviews_holdout['rating'] >reviews_holdout['average_stars'])]
    if len(tmp['user_id'].unique())>4:

        ###this weird sampling technique is to ensure we dont' sample the same user twice in a same city
        five_users = np.random.choice(tmp['user_id'].unique(),5, replace = False)
        row = tmp[tmp['user_id'].isin(five_users)].groupby('user_id', group_keys=False).apply(lambda df: df.sample(1))
        out = out.append(row)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_16920\1404869436.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(row)


In [20]:
predict_df = out[['user_id','city','state']]
predict_df = predict_df.merge(unique_city_businesses, on = 'city')
predict_df.to_csv('../datasets/filter_data/metric_sample.csv')

In [21]:
# random initialization, needs to be substituted by actual predictions later.
predict_df['predictions'] = 2.5

In [22]:
def get_all_metrics(predict_df, validation_subsample, ratings_train_final):
    top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()
    out = validation_subsample
    cnt =0
    serendipity = 0


    for row in out.iterrows():
        row_values = row[1]
        top_10 = predict_df.loc[top_10_recs[top_10_recs['user_id'] == row_values['user_id']].level_2]['business_id']
        ###In top 10
        if row_values['business_id'] in top_10.values:
            cnt+=1
        user_history = ratings_train_final[ratings_train_final['user_id'] == row_values['user_id']]
        been_there = [i for i in top_10.values if i in  user_history.business_id.values]
        serendipity += 1-len(been_there)/10

    top_10 = cnt/len(out)
    serendipity = serendipity/len(out)

    predict_df = predict_df.reset_index()

    analysis_df = predict_df.merge(top_10_recs, left_on = ['user_id','city','index'], \
                                   right_on = ['user_id','city','level_2'])

    coverage = (analysis_df.groupby('city')['business_id'].nunique()/50).values.mean()

    predict_df['rankings']=predict_df.groupby(['city','user_id'])['predictions']. \
        rank(method="first",ascending = False)
    running_rankings =0
    for row in out.iterrows():
        row_values = row[1]
        user_recs = predict_df[(predict_df['user_id']==row_values['user_id'])
                               &(predict_df['city']==row_values['city'])
                               & (predict_df['business_id']==row_values['business_id'])
                               ]
        assert len(user_recs)==1
        running_rankings += user_recs['rankings'].sum()

    avg_rank = running_rankings / len(out)
    print(top_10, coverage, serendipity, avg_rank)

    return top_10, coverage, serendipity, avg_rank


Baseline models
Baseline 1: Bias Baseline

In [23]:
from surprise import SVD
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import BaselineOnly

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [24]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(df_res_bus, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                   'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

reviews_train = process(reviews_train_.copy())
reviews_test = process(reviews_holdout_.copy())
reviews_val = process(reviews_cv_.copy())

In [25]:
# remove observations that may cause cold-start problem, which breaks the model.
reviews_test = reviews_test.loc[reviews_test.business_id.isin(reviews_train.business_id)]
reviews_val = reviews_val.loc[reviews_val.business_id.isin(reviews_train.business_id)]

In [26]:
trainset = reviews_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = reviews_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = reviews_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [27]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

In [28]:
bsl_options = {'method': 'als', 'n_epochs':3}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.mae(predictions)
accuracy.rmse(predictions)

Estimating biases using als...
MAE:  0.8861
RMSE: 1.0836


1.0835728317188933

In [29]:
#predictions_B1=pd.head()

In [30]:
bsl_options = {'method': 'als', 'n_epochs':5}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.mae(predictions)
accuracy.rmse(predictions)

Estimating biases using als...
MAE:  0.8860
RMSE: 1.0836


1.083570397238404

In [31]:
bsl_options = {'method': 'als', 'n_epochs':9}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.mae(predictions)
accuracy.rmse(predictions)

Estimating biases using als...
MAE:  0.8860
RMSE: 1.0836


1.0835703915170394

Observation: It seems different hyperparameters all performs the same result; the team just uses default.

Evaluation
Note: evaluated 20%, 50%, 100% training data respectively. For readability, only result on 20% data is displayed.

In [32]:
# runtime on 20% data
start_time = time.time()
bias_baseline.fit(train_sr)
print("--- %s seconds ---" % (time.time() - start_time))

Estimating biases using als...
--- 0.0 seconds ---


In [33]:
# Regression metrics 
bbase_p = bias_baseline.test(test_sr)
start_time = time.time()
bbase_20_df = pd.DataFrame(bbase_p, columns = ['userId','itemId','rating','pred_rating','x'])
accuracy.rmse(bbase_p)
print('R^2 (with 20% data): ', r2_score(bbase_20_df.rating , bbase_20_df.pred_rating))
print('MAE (with 20% data): ', mean_absolute_error(bbase_20_df.rating, bbase_20_df.pred_rating))
print("--- %s seconds ---" % (time.time() - start_time))

RMSE: 1.1242
R^2 (with 20% data):  -0.09553905286842501
MAE (with 20% data):  0.9339870236252058
--- 0.0 seconds ---


In [34]:
# Ranking + coverage + novelty metrics
predict_df_baseline = predict_df.copy()
eval_set = Dataset.load_from_df(predict_df_baseline[['user_id','business_id','predictions']], reader)

bias_basline = BaselineOnly({'method': 'als', 'n_epochs':9})
eval_before = eval_set.build_full_trainset()
eval_sr = eval_before.build_testset()
bias_basline.fit(train_sr)
eval_pred = bias_basline.test(eval_sr)
#accuracy.rmse(predictions_50)
baseline_20 = pd.DataFrame(eval_pred, columns = ['userId','itemId','rating','pred_rating','x'])
predict_df_baseline['predictions'] = baseline_20.pred_rating

Estimating biases using als...


In [35]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_baseline, out, reviews_train_final)

0.4 0.2 0.9199999999999999 16.6


In [36]:
predict_df_baseline.head(20)

,user_id,city,state,business_id,predictions
0,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,xwKYBPO0ByGlkvNcr8FdqQ,4.237067
1,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,LXR1f2ayOSq7w-Ux7UPUtg,3.765504
2,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,RHdEScVIAQ7xzFFMiQEnAQ,3.949817
3,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,l8yqThofpWGUs6lxOeIdOA,4.183969
4,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,XyGRDrgCK0z4CiA6nhwEaw,4.210074
5,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,5CX7rtnwYnmqFCihOyWivg,4.067190
6,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,aDL_KdxbDRT5s51Jtn9V5g,3.976353
7,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,7PbJjm9Oeq9LqM9zKsF-8g,3.529265
8,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,nEaTbGFlU7d9eLU2kl6KBw,4.043599
9,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,FjRH6l8aQOmfXuo00lheLA,3.724462


In [37]:
top10_baseline=predict_df_baseline.groupby('business_id')["predictions"].mean()
top10_baseline.sort_values(ascending=False).head(10)

business_id
xwKYBPO0ByGlkvNcr8FdqQ    4.140940
CJoO4HYD0tZRXlZqA04wmw    4.134371
dlOTtFJbcyN1m5PR2IDxOg    4.121631
XyGRDrgCK0z4CiA6nhwEaw    4.113947
vtbAuFCR8ugUaajtF_wDEQ    4.103778
l8yqThofpWGUs6lxOeIdOA    4.087842
yRs15dgEYvxGatbDyeVXiA    4.033280
wzPc5d1l0PpxQLQDhh8T2w    4.014900
p7tSGmWpFV246aYhwODgBg    4.006801
5Ce3lZksYVkCbrihqylVHQ    3.997013
Name: predictions, dtype: float64

In [38]:
resultado_baseline=top10_baseline.to_frame()
print(type(resultado_baseline))

<class 'pandas.core.frame.DataFrame'>


In [39]:
df_join_baseline=resultado_baseline.merge(df_res_bus[["business_id","restaurant"]],how="inner",on="business_id")
df_join_baseline.head(10)

,business_id,predictions,restaurant
0,5CX7rtnwYnmqFCihOyWivg,3.971063,Opa's Best Greek American Cuisine
1,5Ce3lZksYVkCbrihqylVHQ,3.997013,Sushi Nara
2,7PbJjm9Oeq9LqM9zKsF-8g,3.433138,Seoul Kitchen
3,A93_XS0xZu1OtfQPTIJeqw,3.829553,Jimmy's Pizzeria
4,AL89lQXFfE0HpbTvpxuSkA,3.910505,Bob's Steak & Chop House
5,AbeWP7_Vxhp45I9IPZ7Otg,3.792694,Mosaic Cafe Dos
6,CJoO4HYD0tZRXlZqA04wmw,4.134371,Crave Coffee Bar
7,FjRH6l8aQOmfXuo00lheLA,3.628336,Broadway Pizza Cafe
8,IJ5N0TNgDDFP0UauRmepDg,3.988634,Luckie's Thai-Asian Cuisine
9,KZA_HEOsBXf8dtrk9rqNJA,3.918936,Prep & Pastry on Grant


In [40]:
df_join_baseline.sort_values(by="predictions", ascending=False)

,business_id,predictions,restaurant
32,xwKYBPO0ByGlkvNcr8FdqQ,4.140940,Le Rendez-vous
6,CJoO4HYD0tZRXlZqA04wmw,4.134371,Crave Coffee Bar
18,dlOTtFJbcyN1m5PR2IDxOg,4.121631,Selena's Salvadorian
15,XyGRDrgCK0z4CiA6nhwEaw,4.113947,Persian Room Fine Dining
30,vtbAuFCR8ugUaajtF_wDEQ,4.103778,New York Pizza
26,l8yqThofpWGUs6lxOeIdOA,4.087842,Shish Kebab House of Tucson
33,yRs15dgEYvxGatbDyeVXiA,4.033280,Costa Vida
31,wzPc5d1l0PpxQLQDhh8T2w,4.014900,Toro Loco
29,p7tSGmWpFV246aYhwODgBg,4.006801,O Sushi Restaurant
1,5Ce3lZksYVkCbrihqylVHQ,3.997013,Sushi Nara


Baseline 2: Collaborative Filtering via SVD

In [41]:
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

In [42]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(df_res_bus, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                   'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

reviews_train = process(reviews_train_.copy())
reviews_holdout = process(reviews_holdout_.copy())
reviews_val = process(reviews_cv_.copy())

In [43]:
# remove observations that may cause cold-start problem, which breaks the model.
reviews_test = reviews_test.loc[reviews_test.business_id.isin(reviews_train.business_id)]
reviews_val = reviews_val.loc[reviews_val.business_id.isin(reviews_train.business_id)]

In [44]:
trainset = reviews_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = reviews_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = reviews_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [45]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

Hyperparameter Tuning

In [46]:
RMSE_tune = {}
n_epochs = [10, 20, 30]  # the number of iteration of the SGD procedure
lr_all = [0.001, 0.003, 0.005] # the learning rate for all parameters
reg_all =  [0.02, 0.05, 0.1, 0.4, 0.5] # the regularization term for all parameters

for n in n_epochs:
    for l in lr_all:
        for r in reg_all:
            print('Fitting n: {0}, l: {1}, r: {2}'.format(n, l, r))
            algo = SVD(n_epochs = n, lr_all = l, reg_all = r)
            algo.fit(train_sr)
            predictions = algo.test(val_sr)
            RMSE_tune[n,l,r] = accuracy.rmse(predictions)

Fitting n: 10, l: 0.001, r: 0.02
RMSE: 1.0966
Fitting n: 10, l: 0.001, r: 0.05
RMSE: 1.1178
Fitting n: 10, l: 0.001, r: 0.1
RMSE: 1.1517
Fitting n: 10, l: 0.001, r: 0.4
RMSE: 1.1057
Fitting n: 10, l: 0.001, r: 0.5
RMSE: 1.0981
Fitting n: 10, l: 0.003, r: 0.02
RMSE: 1.0915
Fitting n: 10, l: 0.003, r: 0.05
RMSE: 1.0810
Fitting n: 10, l: 0.003, r: 0.1
RMSE: 1.0831
Fitting n: 10, l: 0.003, r: 0.4
RMSE: 1.0900
Fitting n: 10, l: 0.003, r: 0.5
RMSE: 1.0986
Fitting n: 10, l: 0.005, r: 0.02
RMSE: 1.0897
Fitting n: 10, l: 0.005, r: 0.05
RMSE: 1.0809
Fitting n: 10, l: 0.005, r: 0.1
RMSE: 1.0951
Fitting n: 10, l: 0.005, r: 0.4
RMSE: 1.0701
Fitting n: 10, l: 0.005, r: 0.5
RMSE: 1.0848
Fitting n: 20, l: 0.001, r: 0.02
RMSE: 1.1095
Fitting n: 20, l: 0.001, r: 0.05
RMSE: 1.0845
Fitting n: 20, l: 0.001, r: 0.1
RMSE: 1.1228
Fitting n: 20, l: 0.001, r: 0.4
RMSE: 1.1150
Fitting n: 20, l: 0.001, r: 0.5
RMSE: 1.1034
Fitting n: 20, l: 0.003, r: 0.02
RMSE: 1.0875
Fitting n: 20, l: 0.003, r: 0.05
RMSE: 1.0929


In [47]:
import operator
min(RMSE_tune.items(), key=operator.itemgetter(1))[0]

(30, 0.005, 0.05)

Evaluation

In [67]:
# train and test on the optimal parameter
start_time = time.time()
algo_real = SVD(n_epochs = 20, lr_all = 0.005, reg_all = 0.05)
algo_real.fit(train_sr)
predictions = algo_real.test(test_sr)

In [68]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.015628576278686523 seconds ---


In [69]:
accuracy.rmse(predictions)

RMSE: 1.0964


1.096364475574629

In [70]:
accuracy.mae(predictions)

MAE:  0.9123


0.9122662605605533

In [71]:
r2_score([t[2] for t in predictions], [t[3] for t in predictions])

-0.041890923202276165

In [72]:
# To evaluate coverage and serendipity metrics, use evaluation set created earlier.
predict_df_20 = pd.read_csv('../datasets/filter_data/metric_sample.csv', index_col=0)
predict_df_20['predictions'] = 2.5 # fill in this value temporally
eval_20 = Dataset.load_from_df(predict_df_20[['user_id','business_id','predictions']], reader)

In [73]:
eval_before_20 = eval_20.build_full_trainset()
eval_sr_20 = eval_before_20.build_testset()
eval_pred_20 = algo_real.test(eval_sr_20)

baseline_20 = pd.DataFrame(eval_pred_20, columns = ['userId','business_id','rating','pred_rating','x'])
predict_df_20['predictions'] = baseline_20.pred_rating

In [74]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_20, out, reviews_train_final)

0.2 0.36 0.86 13.2


In [75]:
predict_df_20.head(20)

,user_id,city,state,business_id,predictions
0,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,xwKYBPO0ByGlkvNcr8FdqQ,4.118759
1,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,LXR1f2ayOSq7w-Ux7UPUtg,3.615065
2,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,RHdEScVIAQ7xzFFMiQEnAQ,4.152578
3,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,l8yqThofpWGUs6lxOeIdOA,4.265540
4,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,XyGRDrgCK0z4CiA6nhwEaw,4.067497
5,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,5CX7rtnwYnmqFCihOyWivg,4.296358
6,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,aDL_KdxbDRT5s51Jtn9V5g,4.098340
7,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,7PbJjm9Oeq9LqM9zKsF-8g,3.652157
8,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,nEaTbGFlU7d9eLU2kl6KBw,4.269680
9,8dUbErn76KcD2Sj_iMOWag,Tucson,AZ,FjRH6l8aQOmfXuo00lheLA,3.810833


In [76]:
top10=predict_df_20.groupby('business_id')["predictions"].mean()
top10.sort_values(ascending=False).head(10)

business_id
CJoO4HYD0tZRXlZqA04wmw    4.219513
vtbAuFCR8ugUaajtF_wDEQ    4.143861
xwKYBPO0ByGlkvNcr8FdqQ    4.130504
l8yqThofpWGUs6lxOeIdOA    4.097425
dlOTtFJbcyN1m5PR2IDxOg    4.092477
IJ5N0TNgDDFP0UauRmepDg    4.085914
p7tSGmWpFV246aYhwODgBg    4.058259
yRs15dgEYvxGatbDyeVXiA    4.050507
XyGRDrgCK0z4CiA6nhwEaw    4.022851
jLnIvbbXUg4s7CP9G-Whvw    4.018111
Name: predictions, dtype: float64

In [77]:
resultado=top10.to_frame()
print(type(resultado))

<class 'pandas.core.frame.DataFrame'>


In [78]:
df_join=resultado.merge(df_res_bus[["business_id","restaurant"]],how="inner",on="business_id")
df_join.head(10)

,business_id,predictions,restaurant
0,5CX7rtnwYnmqFCihOyWivg,3.968531,Opa's Best Greek American Cuisine
1,5Ce3lZksYVkCbrihqylVHQ,3.911790,Sushi Nara
2,7PbJjm9Oeq9LqM9zKsF-8g,3.450924,Seoul Kitchen
3,A93_XS0xZu1OtfQPTIJeqw,3.791771,Jimmy's Pizzeria
4,AL89lQXFfE0HpbTvpxuSkA,3.901890,Bob's Steak & Chop House
5,AbeWP7_Vxhp45I9IPZ7Otg,3.778322,Mosaic Cafe Dos
6,CJoO4HYD0tZRXlZqA04wmw,4.219513,Crave Coffee Bar
7,FjRH6l8aQOmfXuo00lheLA,3.631218,Broadway Pizza Cafe
8,IJ5N0TNgDDFP0UauRmepDg,4.085914,Luckie's Thai-Asian Cuisine
9,KZA_HEOsBXf8dtrk9rqNJA,3.885965,Prep & Pastry on Grant


In [79]:
df_join.sort_values(by="predictions", ascending=False)

,business_id,predictions,restaurant
6,CJoO4HYD0tZRXlZqA04wmw,4.219513,Crave Coffee Bar
30,vtbAuFCR8ugUaajtF_wDEQ,4.143861,New York Pizza
32,xwKYBPO0ByGlkvNcr8FdqQ,4.130504,Le Rendez-vous
26,l8yqThofpWGUs6lxOeIdOA,4.097425,Shish Kebab House of Tucson
18,dlOTtFJbcyN1m5PR2IDxOg,4.092477,Selena's Salvadorian
8,IJ5N0TNgDDFP0UauRmepDg,4.085914,Luckie's Thai-Asian Cuisine
29,p7tSGmWpFV246aYhwODgBg,4.058259,O Sushi Restaurant
33,yRs15dgEYvxGatbDyeVXiA,4.050507,Costa Vida
15,XyGRDrgCK0z4CiA6nhwEaw,4.022851,Persian Room Fine Dining
24,jLnIvbbXUg4s7CP9G-Whvw,4.018111,Harvest on River


In [80]:
print(baseline_20)

                     userId             business_id  rating  pred_rating  \
0    8dUbErn76KcD2Sj_iMOWag  xwKYBPO0ByGlkvNcr8FdqQ     2.5     4.118759   
1    8dUbErn76KcD2Sj_iMOWag  LXR1f2ayOSq7w-Ux7UPUtg     2.5     3.615065   
2    8dUbErn76KcD2Sj_iMOWag  RHdEScVIAQ7xzFFMiQEnAQ     2.5     4.152578   
3    8dUbErn76KcD2Sj_iMOWag  l8yqThofpWGUs6lxOeIdOA     2.5     4.265540   
4    8dUbErn76KcD2Sj_iMOWag  XyGRDrgCK0z4CiA6nhwEaw     2.5     4.067497   
..                      ...                     ...     ...          ...   
165  N6ZTMIue-2b30CJv2tyPGg  A93_XS0xZu1OtfQPTIJeqw     2.5     3.824168   
166  N6ZTMIue-2b30CJv2tyPGg  a_oRfgkNFnulTada--P_Tg     2.5     3.906190   
167  N6ZTMIue-2b30CJv2tyPGg  eGi7CE2YaN6YMiGyXNi8XQ     2.5     4.027764   
168  N6ZTMIue-2b30CJv2tyPGg  g9DDXMuDwaW-GtD2f6xl6A     2.5     4.027764   
169  N6ZTMIue-2b30CJv2tyPGg  AL89lQXFfE0HpbTvpxuSkA     2.5     4.027764   

                             x  
0    {'was_impossible': False}  
1    {'was_impossible

In [81]:
# Define a custom sorting key based on the rating
def pred_rating (baseline_20):
    return baseline_20["pred_rating"]

In [82]:
top10_2=baseline_20.groupby('business_id')["pred_rating"].mean()
top10_2.sort_values(ascending=False).head(10)

business_id
CJoO4HYD0tZRXlZqA04wmw    4.219513
vtbAuFCR8ugUaajtF_wDEQ    4.143861
xwKYBPO0ByGlkvNcr8FdqQ    4.130504
l8yqThofpWGUs6lxOeIdOA    4.097425
dlOTtFJbcyN1m5PR2IDxOg    4.092477
IJ5N0TNgDDFP0UauRmepDg    4.085914
p7tSGmWpFV246aYhwODgBg    4.058259
yRs15dgEYvxGatbDyeVXiA    4.050507
XyGRDrgCK0z4CiA6nhwEaw    4.022851
jLnIvbbXUg4s7CP9G-Whvw    4.018111
Name: pred_rating, dtype: float64

In [83]:
resultado_2=top10_2.to_frame()
print(type(resultado_2))

<class 'pandas.core.frame.DataFrame'>


In [84]:
df_join_2=resultado_2.merge(df_res_bus[["business_id","restaurant"]],how="inner",on="business_id")
df_join_2.head(10)

,business_id,pred_rating,restaurant
0,5CX7rtnwYnmqFCihOyWivg,3.968531,Opa's Best Greek American Cuisine
1,5Ce3lZksYVkCbrihqylVHQ,3.911790,Sushi Nara
2,7PbJjm9Oeq9LqM9zKsF-8g,3.450924,Seoul Kitchen
3,A93_XS0xZu1OtfQPTIJeqw,3.791771,Jimmy's Pizzeria
4,AL89lQXFfE0HpbTvpxuSkA,3.901890,Bob's Steak & Chop House
5,AbeWP7_Vxhp45I9IPZ7Otg,3.778322,Mosaic Cafe Dos
6,CJoO4HYD0tZRXlZqA04wmw,4.219513,Crave Coffee Bar
7,FjRH6l8aQOmfXuo00lheLA,3.631218,Broadway Pizza Cafe
8,IJ5N0TNgDDFP0UauRmepDg,4.085914,Luckie's Thai-Asian Cuisine
9,KZA_HEOsBXf8dtrk9rqNJA,3.885965,Prep & Pastry on Grant


In [85]:
df_join_2.sort_values(by="pred_rating", ascending=False)

,business_id,pred_rating,restaurant
6,CJoO4HYD0tZRXlZqA04wmw,4.219513,Crave Coffee Bar
30,vtbAuFCR8ugUaajtF_wDEQ,4.143861,New York Pizza
32,xwKYBPO0ByGlkvNcr8FdqQ,4.130504,Le Rendez-vous
26,l8yqThofpWGUs6lxOeIdOA,4.097425,Shish Kebab House of Tucson
18,dlOTtFJbcyN1m5PR2IDxOg,4.092477,Selena's Salvadorian
8,IJ5N0TNgDDFP0UauRmepDg,4.085914,Luckie's Thai-Asian Cuisine
29,p7tSGmWpFV246aYhwODgBg,4.058259,O Sushi Restaurant
33,yRs15dgEYvxGatbDyeVXiA,4.050507,Costa Vida
15,XyGRDrgCK0z4CiA6nhwEaw,4.022851,Persian Room Fine Dining
24,jLnIvbbXUg4s7CP9G-Whvw,4.018111,Harvest on River
